#Digital Patient Feedback & Experience Agent

###Overview


This Digital Patient Feedback & Experience Agent is designed to automate the collection, analysis, and tracking of patient feedback in healthcare. It uses Pydantic for data validation, OpenTelemetry for observability, and Open AI GPT for sentiment analysis, trend detection, and issue tracking.

This agent ensures hospitals receive structured feedback, detect service quality trends, and are alerted to urgent patient concerns.

###Core Functionalities & Workflow

###Step 1: Collect Patient Feedback

*   Patients submit structured feedback, including ID, name, date, topic, rating, and comments.
*   Pydantic Validation ensures only correctly formatted data is accepted.
*   Multi-turn conversations clarify vague feedback by asking follow-up questions.

###Step 2: Sentiment Analysis

*   AI classifies feedback as Positive, Neutral, or Negative.
*   Observability (OpenTelemetry) logs how AI classified sentiment for tracking.

###Step 3: Trend Analysis

*   Detects recurring issues & improvement areas in patient feedback.
*   Past trends are stored in feedback_trends.json to track service quality over time.

### Step 4: Issue Tracking

*   If a negative sentiment is detected, it extracts critical concerns.
*   Issues such as long wait times, unclear billing, or rude staff are flagged.

###Step 5: Notifications & Alerts

*   If critical issues exist, an email alert is sent to hospital administrators.
*   The AI system ensures only serious concerns trigger alerts (dynamic execution).

###Step 6: Final Report

The report includes:

✔ Sentiment Breakdown (positive, neutral, negative)

✔ Identified Trends (e.g., increasing wait time complaints)

✔ Flagged Issues (e.g., billing concerns)

✔ Notifications Sent (if any critical alerts were raised)

Observability logs track report generation for transparency.







###Set Up

In [ ]:
!pip install pydantic openai opentelemetry-api opentelemetry-sdk opentelemetry-instrumentation-logging opentelemetry-exporter-otlp python-dotenv


pydantic → Validates and structures patient feedback data.

openai → Enables AI-powered sentiment analysis, trends, and notifications.

opentelemetry-sdk → Monitors agent execution and logs performance.

opentelemetry-instrumentation-logging → Tracks logs and debugging info.

###Patient Feedback Schema

Using Pydantic's BaseModel to enforce structured input for patient feedback

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional


class PatientFeedback(BaseModel):
    patient_id: int = Field(..., example=101, description="Unique identifier for the patient.")
    patient_name: str = Field(..., example="John Doe", description="Name of the patient.")
    date: str = Field(..., example="2025-03-25", description="Date when feedback was given (YYYY-MM-DD).")
    subject: str = Field(..., example="Billing Issue", description="Feedback category (e.g., Billing, Waiting Time).")
    satisfaction_rating: int = Field(..., ge=1, le=5, example=3, description="Rating (1-5) based on experience.")
    comments: str = Field(..., example="The billing process was confusing.", description="Detailed feedback.")

print("Pydantic Patient Feedback Model Defined!")


Pydantic Patient Feedback Model Defined!


###Set Up OpenTelemetry for Logging

Ensures observability of AI agent processing and tracks every function execution (e.g., Sentiment Analysis, Trends, Issue Tracking).

In [ ]:
import logging
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from opentelemetry.instrumentation.logging import LoggingInstrumentor

LoggingInstrumentor().instrument() #Enable logging instrumentation

trace.set_tracer_provider(TracerProvider())
tracer = trace.get_tracer(__name__)
span_processor = SimpleSpanProcessor(ConsoleSpanExporter()) #  Configure Console Exporter (log traces locally)
trace.get_tracer_provider().add_span_processor(span_processor)

print(" OpenTelemetry done!")

 OpenTelemetry done!


Detects vague feedback & asks follow-up questions

In [ ]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

def clarify_feedback(feedback_text):
    """Engages in multi-turn conversations to clarify vague feedback before processing."""
    with tracer.start_as_current_span("Clarifying Feedback"):
        clarification_rounds = 2  # follow-ups to 2 qns
        refined_feedback = feedback_text

        for _ in range(clarification_rounds):
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an AI that detects vague patient feedback and suggests follow-up questions."},
                    {"role": "user", "content": f"Does this feedback need clarification? If yes, suggest a follow-up question: {refined_feedback}"}
                ]
            )
            follow_up_question = response.choices[0].message.content.strip()

            if "No clarification needed" in follow_up_question:
                break

            print(f"\n🤖 **Follow-Up Question:** {follow_up_question}")
            additional_details = input("💬 Your Response: ")

            refined_feedback += f" {additional_details}"

        return refined_feedback



###Create Patient Memory Storage

Stores: Each patient’s past feedback, sentiment, and complaints.

In [ ]:
import json
import os
PATIENT_MEMORY_FILE = "patient_feedback_history.json"

def load_patient_memory():
    """Loads stored patient feedback history from memory."""
    if os.path.exists(PATIENT_MEMORY_FILE):
        with open(PATIENT_MEMORY_FILE, "r") as file:
            return json.load(file)
    return {}

def save_patient_memory(patient_id, feedback_entry):
    """Stores feedback history for a specific patient."""
    memory = load_patient_memory()

    if str(patient_id) not in memory:
        memory[str(patient_id)] = []

    memory[str(patient_id)].append(feedback_entry)
    with open(PATIENT_MEMORY_FILE, "w") as file:
        json.dump(memory, file, indent=4)

###Collect Patient Feedback (Validated with Pydantic and log it with OpenTelemetry)

In [ ]:
def collect_patient_feedback():
    """Collects patient feedback interactively, validates input using Pydantic,
    asks follow-up questions if needed, and remembers past patient issues."""

    feedback_list = []

    while True:
        try:
            print("\n💬 **Patient Feedback Collection**")
            with tracer.start_as_current_span("Feedback Collection"):

                patient_id = input("Enter Patient ID (or type 'done' to finish): ")
                if patient_id.lower() == "done":
                    break

                patient_id = int(patient_id)
                patient_name = input("Enter Patient Name: ")
                date = input("Enter Feedback Date (YYYY-MM-DD): ")
                subject = input("Enter Subject/Topic (e.g., 'Billing', 'Waiting Time', etc.): ")
                satisfaction_rating = int(input("Enter Satisfaction Rating (1-5): "))
                comments = input("Enter Patient Comments: ")

                past_memory = load_patient_memory()
                if str(patient_id) in past_memory:
                    past_issues = [entry["comments"] for entry in past_memory[str(patient_id)]]
                    print(f"\n **Previous Feedback from {patient_name}:** {past_issues}")

                follow_up_question = clarify_feedback(comments)
                if follow_up_question:
                    print(f"\n🤖 **Follow-Up Question:** {follow_up_question}")
                    additional_details = input("💬 Your Response: ")
                    comments += " " + additional_details

                # Validate feedback using Pydantic
                feedback = PatientFeedback(
                    patient_id=patient_id,
                    patient_name=patient_name,
                    date=date,
                    subject=subject,
                    satisfaction_rating=satisfaction_rating,
                    comments=comments
                )

                feedback_list.append(feedback.model_dump())
                logging.info(f"✅ Feedback Recorded: {feedback.model_dump()}")

                # Save Feedback in Long-Term Memory
                save_patient_memory(patient_id, feedback.model_dump())

                print("\n Feedback Recorded Successfully!\n")

        except ValueError:
            print(" Invalid Input! Please enter numbers for Patient ID and Satisfaction Rating.")
        except Exception as e:
            print(f" Error: {e}")
        finally:
            if input(" Add another feedback? (yes/no): ").lower() == "no":
                break

    return feedback_list


###Run the Agent to Collect Feedback

In [ ]:

collected_feedback = collect_patient_feedback()

print("\n **Structured Patient Feedback Data:**")
for feedback in collected_feedback:
    print(feedback)



💬 **Patient Feedback Collection**
Enter Patient ID (or type 'done' to finish): 6
Enter Patient Name: Zuck
Enter Feedback Date (YYYY-MM-DD): 2025-03-16
Enter Subject/Topic (e.g., 'Billing', 'Waiting Time', etc.): Time
Enter Satisfaction Rating (1-5): 2
Enter Patient Comments: time taking

🤖 **Follow-Up Question:** Yes, this feedback needs clarification. A follow-up question could be: "Could you please specify which aspects of the service you found to be time-consuming?"
💬 Your Response: waiting time was too long

🤖 **Follow-Up Question:** Yes, this feedback needs clarification. 

Follow-up question: "Could you specify how long you waited, and which part of the process took the most time?"
💬 Your Response: for 3 hrs and that too for appointment
{
    "name": "Clarifying Feedback",
    "context": {
        "trace_id": "0x97a3ad0e1251b12637e131a460eacb61",
        "span_id": "0x01153c5782ef6b7f",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xa2b57

###Implement Functions for Feedback Analysis

In [ ]:
import openai
import os
import json
from datetime import datetime
import smtplib
import os
from email.message import EmailMessage
from dotenv import load_dotenv
load_dotenv()

EMAIL_SENDER = os.getenv("EMAIL_SENDER")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
EMAIL_RECEIVER = os.getenv("EMAIL_RECEIVER")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

TREND_MEMORY_FILE = "feedback_trends.json" #Store past feedback trends in a JSON file

def load_past_trends():
    """Loads past trend data from the memory file."""
    if os.path.exists(TREND_MEMORY_FILE):
        with open(TREND_MEMORY_FILE, "r") as file:
            return json.load(file)
    return {"trends": [], "last_updated": "N/A"}

def save_trends(new_trends):
    """Updates the trend memory with new trend data."""
    data = {
        "trends": new_trends,
        "last_updated": datetime.now().strftime("%Y-%m-%d")
    }
    with open(TREND_MEMORY_FILE, "w") as file:
        json.dump(data, file, indent=4)


# 1. Sentiment Analysis
def analyze_sentiment(feedback_text):
    """Uses OpenAI API to classify sentiment as Positive, Neutral, or Negative."""
    with tracer.start_as_current_span("Sentiment Analysis"):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI that classifies sentiment of patient feedback as Positive, Neutral, or Negative."},
                {"role": "user", "content": f"Classify this feedback: {feedback_text}"}
            ]
        )

        sentiment = response.choices[0].message.content.strip()
        logging.info(f" Sentiment Analysis: {sentiment}")
        return sentiment

# 2.Trend Analysis (With Memory)
def analyze_trends(feedback_list):
    """Identifies recurring themes from multiple feedbacks, considering past trends."""
    with tracer.start_as_current_span("Trend Analysis"):

        past_trends = load_past_trends()["trends"] # Load past trends
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an AI that detects trends in patient feedback, considering past trends."},
                {"role": "user", "content": f"Given these past trends: {past_trends}, identify new trends in this feedback: {feedback_list}"}
            ]
        ) #trend analysis now remembers past trends and compares them with new ones.
        new_trends = response.choices[0].message.content.strip()

        save_trends(new_trends)

        logging.info(f" Updated Trend Analysis: {new_trends}")
        return new_trends

#3. Track Issues
def track_issues(feedback_list):
    """Detects patient concerns, checking if they are repeated issues from past feedback."""
    with tracer.start_as_current_span("Issue Tracking"):

        flagged_issues = []
        patient_memory = load_patient_memory()  # Load past feedback data

        for feedback in feedback_list:
            patient_id = str(feedback["patient_id"])
            comments = feedback["comments"]

            # Check if this complaint was made before
            if patient_id in patient_memory:
                past_comments = [entry["comments"] for entry in patient_memory[patient_id]]
                if comments in past_comments:
                    flagged_issues.append(f"⚠ **Repeated Issue** from Patient {patient_id}: {comments}")
                else:
                    flagged_issues.append(f"🔹 **New Issue** from Patient {patient_id}: {comments}")
            else:
                flagged_issues.append(f"🔹 **New Issue** from Patient {patient_id}: {comments}")

        if flagged_issues:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an AI that prioritizes urgent patient concerns."},
                    {"role": "user", "content": f"Summarize these flagged issues: {flagged_issues}"}
                ]
            )
            summary = response.choices[0].message.content.strip()
            logging.info(f" Issue Tracking Summary: {summary}")
            return summary

        return " No urgent issues detected"

#4. Notifications
def send_notifications(issues):
    """Generates alerts and emails the hospital authority about patient concerns."""
    with tracer.start_as_current_span("Notifications"):
        if not issues or issues == "No urgent issues detected":
            return "No critical issues detected."

        notification_message = f"🚨 ALERT: Immediate action required for these patient concerns:\n\n{issues}"
        logging.info(f" Notification Sent: {notification_message}")

        send_email_alert(notification_message)

        return notification_message

# 5. Helper Function
def send_email_alert(issue_details):
    """Sends an email notification with the flagged patient issues."""
    try:
        msg = EmailMessage()
        msg.set_content(f"""
        📢 **Hospital Alert - Urgent Patient Concerns**

        Dear Hospital Administration,

        The following patient concerns require **immediate attention**:

        {issue_details}

        Please review the situation and take necessary actions.

        Best Regards,
        **AI Feedback Monitoring System**
        """)

        msg["Subject"] = "🚨 URGENT: Patient Feedback Alert"
        msg["From"] = EMAIL_SENDER
        msg["To"] = EMAIL_RECEIVER


        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server: # Establish SMTP connection
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(" Email Alert Sent!")
        print(" Email notification sent to hospital administration!")

    except Exception as e:
        logging.error(f" Failed to send email alert: {e}")
        print(" Email sending failed. Check SMTP settings.")

print(" AI Feedback Analysis Functions Implemented Successfully!")


 AI Feedback Analysis Functions Implemented Successfully!


###Run AI Analysis on Collected Feedback

In [ ]:
for feedback in collected_feedback:
    print("\n📝 **Patient Feedback:**", feedback["comments"])

    sentiment_result = analyze_sentiment(feedback["comments"])
    print(f"🔍 **Sentiment Analysis:** {sentiment_result}")

trend_result = analyze_trends([fb["comments"] for fb in collected_feedback])
print(f"\n📊 **Trend Analysis:** {trend_result}")

issue_result = track_issues(collected_feedback)
print(f"\n⚠ **Flagged Issues:** {issue_result}")

notification_result = send_notifications(issue_result)
print(f"\n🚨 **Notifications:** {notification_result}")



📝 **Patient Feedback:** time taking yes
{
    "name": "Sentiment Analysis",
    "context": {
        "trace_id": "0x7550889a0db84b2dfecde527a4de3650",
        "span_id": "0xd3b364533ff06a75",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-03-19T10:54:24.178818Z",
    "end_time": "2025-03-19T10:54:24.678621Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.31.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "Sentiment Analysis",
    "context": {
        "trace_id": "0x7550889a0db84b2dfecde527a4de3650",
        "span_id": "0xd3b364533ff06a75",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent

ERROR:root: Failed to send email alert: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 71dfb90a1353d-5243a6e5432sm2380320e0c.34 - gsmtp')


 Email sending failed. Check SMTP settings.
{
    "name": "Notifications",
    "context": {
        "trace_id": "0xce45fc726d92b3650fccd44221bf9925",
        "span_id": "0xf5b7f95169faa86b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-03-19T10:54:31.659474Z",
    "end_time": "2025-03-19T10:54:31.892678Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.31.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "Notifications",
    "context": {
        "trace_id": "0xce45fc726d92b3650fccd44221bf9925",
        "span_id": "0xf5b7f95169faa86b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": n

###Report Generation Function


Intelligent tool selection:
*   Analyze feedback first and decide which tools to use.
*   Only run Issue Tracking if there is negative sentiment.
*    Only run Trend Analysis if we have at least 3+ feedbacks (so it’s meaningful).
*   Only send Notifications if critical issues exist.

In [ ]:
def generate_patient_feedback_report(feedback_list):
    """Generates a structured final report for hospital staff based on AI analysis."""

    print("\n===  **Final Patient Feedback Report** ===\n")
    past_trends = load_past_trends()

    print(f"\n📈 **Past Trends:** {past_trends['trends']} (Last Updated: {past_trends['last_updated']})")
    sentiment_counts = {"Positive": 0, "Neutral": 0, "Negative": 0}# Track sentiment counts

    detailed_analysis = []
    flagged_issues = []
    feedback_texts = [fb["comments"] for fb in feedback_list]

    # First Pass - Sentiment Analysis for Each Feedback
    for feedback in feedback_list:
        patient_id = feedback["patient_id"]
        patient_name = feedback["patient_name"]
        feedback_text = feedback["comments"]

        print(f"\n**Patient ID:** {patient_id} | **Name:** {patient_name}")
        print(f" **Feedback:** {feedback_text}")

        # Run Sentiment Analysis
        sentiment = analyze_sentiment(feedback_text)
        sentiment_counts[sentiment] += 1
        print(f" **Sentiment:** {sentiment}")

        detailed_analysis.append({
            "Patient ID": patient_id,
            "Patient Name": patient_name,
            "Feedback": feedback_text,
            "Sentiment": sentiment,
        })

        if sentiment == "Negative":
            flagged_issues.append(feedback_text)

    # Dynamic tooling
    run_issue_tracking = len(flagged_issues) > 0
    run_trend_analysis = len(feedback_list) > 3
    run_notifications = run_issue_tracking

    trend_analysis = analyze_trends(feedback_texts) if run_trend_analysis else past_trends["trends"]
    issue_tracking = track_issues([fb for fb in feedback_list if fb["comments"] in flagged_issues]) if run_issue_tracking else "No urgent issues detected"
    notification_alerts = send_notifications(issue_tracking) if run_issue_tracking else "No notifications sent"

    print("\n📊 **Overall Sentiment Breakdown:**")
    for key, value in sentiment_counts.items():
        print(f"- {key}: {value}")

    print("\n📈 **Updated Trend Analysis:**", trend_analysis)
    print("\n⚠ **Flagged Issues:**", issue_tracking)
    print("\n🚨 **Notifications Sent:**", notification_alerts)

    return {
        "Detailed Patient Analysis": detailed_analysis,
        "Overall Sentiment Breakdown": sentiment_counts,
        "Updated Trends": trend_analysis,
        "Flagged Issues": issue_tracking,
        "Notifications": notification_alerts
    }

print("Done!")


Done!


###Execute and Generate the Final Report

Run the full pipeline on our collected feedback

In [ ]:
final_report = generate_patient_feedback_report(collected_feedback)

import json
print("\n=== 📑 **Final Report (JSON Format)** ===\n")
print(json.dumps(final_report, indent=4))



===  **Final Patient Feedback Report** ===


📈 **Past Trends:** Based on the feedback "time taking yes" and considering the past trends related to long waiting times, here are some new or evolving trends that might emerge:

1. **Integrated Live Updates**: Building on proactive patient engagement, there may be an increase in integration of live updates through platforms patients frequently use, such as transportation apps or smart home devices, to keep them informed about wait times or changes with minimal effort.

2. **Virtual Pre-Consultations**: The rise of digital health solutions may lead to an increase in virtual pre-consultations, where routine checks and preliminary assessments are conducted online, reducing the time needed for in-person visits.

3. **Patient Autonomy in Scheduling**: Empowering patients to have more control over their appointments could become a trend. Systems might allow patients to see and select available slots based on their convenience and shortest waitin

ERROR:root: Failed to send email alert: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials a1e0cc1a2514c-86d90d68eb2sm2441954241.7 - gsmtp')


 Email sending failed. Check SMTP settings.
{
    "name": "Notifications",
    "context": {
        "trace_id": "0x4d6e546354d653de99b6fae5bf90493b",
        "span_id": "0x1169abdfeb112c08",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-03-19T10:55:19.082015Z",
    "end_time": "2025-03-19T10:55:19.255880Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.31.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "Notifications",
    "context": {
        "trace_id": "0x4d6e546354d653de99b6fae5bf90493b",
        "span_id": "0x1169abdfeb112c08",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": n

In [ ]:
with open("feedback_trends.json", "r") as file:
    data = json.load(file)
    print("📁 Stored Trends:", json.dumps(data, indent=4))


📁 Stored Trends: {
    "trends": "Based on the feedback \"time taking yes\" and considering the past trends related to long waiting times, here are some new or evolving trends that might emerge:\n\n1. **Integrated Live Updates**: Building on proactive patient engagement, there may be an increase in integration of live updates through platforms patients frequently use, such as transportation apps or smart home devices, to keep them informed about wait times or changes with minimal effort.\n\n2. **Virtual Pre-Consultations**: The rise of digital health solutions may lead to an increase in virtual pre-consultations, where routine checks and preliminary assessments are conducted online, reducing the time needed for in-person visits.\n\n3. **Patient Autonomy in Scheduling**: Empowering patients to have more control over their appointments could become a trend. Systems might allow patients to see and select available slots based on their convenience and shortest waiting times.\n\n4. **Predic